# Predicción de series temporales univariantes


A continuación vamos a entrenar un modelo de series temporales univariantes para predecir el número de pasajeros de una aerolínea. Código basado en la documentación oficial de pycaret https://github.com/pycaret/pycaret y el artículo de Moez Ali https://moez-62905.medium.com/

# Parte I

### Instalación de librerias

In [1]:
#!pip install pycaret==2.2.3
#!pip install -U scikit-learn==0.23.2

     ---------------------------------------- 0.0/249.9 kB ? eta -:--:--
     --------- ----------------------------- 61.4/249.9 kB 3.4 MB/s eta 0:00:01
     -------------------------------------  245.8/249.9 kB 3.8 MB/s eta 0:00:01
     -------------------------------------- 249.9/249.9 kB 3.1 MB/s eta 0:00:00
     ---------------------------------------- 0.0/61.2 kB ? eta -:--:--
     ---------------------------------------- 61.2/61.2 kB ? eta 0:00:00
  Using cached joblib-1.3.2-py3-none-any.whl.metadata (5.4 kB)
     ---------------------------------------- 0.0/7.2 MB ? eta -:--:--
     - -------------------------------------- 0.3/7.2 MB 9.9 MB/s eta 0:00:01
     ---- ----------------------------------- 0.7/7.2 MB 9.2 MB/s eta 0:00:01
     ------ --------------------------------- 1.1/7.2 MB 8.8 MB/s eta 0:00:01
     -------- ------------------------------- 1.5/7.2 MB 8.6 MB/s eta 0:00:01
     ---------- ----------------------------- 1.9/7.2 MB 9.2 MB/s eta 0:00:01
     ------------ 

  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> [2915 lines of output]
      Ignoring numpy: markers 'python_version == "3.6" and platform_system != "AIX" and platform_python_implementation == "CPython"' don't match your environment
      Ignoring numpy: markers 'python_version == "3.6" and platform_system != "AIX" and platform_python_implementation != "CPython"' don't match your environment
      Ignoring numpy: markers 'python_version == "3.7" and platform_system != "AIX"' don't match your environment
      Ignoring numpy: markers 'python_version == "3.6" and platform_system == "AIX"' don't match your environment
      Ignoring numpy: markers 'python_version == "3.7" and platform_system == "AIX"' don't match your environment
      Ignoring numpy: markers 'python_version >= "3.8" and platform_system == "AIX"' don't match your environment
           ---------------------------------------- 0.0/6.

  Using cached scikit-learn-0.23.2.tar.gz (7.2 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'error'


  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> [2066 lines of output]
      Ignoring numpy: markers 'python_version == "3.6" and platform_system != "AIX" and platform_python_implementation == "CPython"' don't match your environment
      Ignoring numpy: markers 'python_version == "3.6" and platform_system != "AIX" and platform_python_implementation != "CPython"' don't match your environment
      Ignoring numpy: markers 'python_version == "3.7" and platform_system != "AIX"' don't match your environment
      Ignoring numpy: markers 'python_version == "3.6" and platform_system == "AIX"' don't match your environment
      Ignoring numpy: markers 'python_version == "3.7" and platform_system == "AIX"' don't match your environment
      Ignoring numpy: markers 'python_version >= "3.8" and platform_system == "AIX"' don't match your environment
        Using cached setuptools-69.0.3-py3-none-any.whl.me

In [2]:
#!pip install plotly==5.1.0 
#!pip install plotly-express==0.4.1

     ---------------------------------------- 0.0/20.6 MB ? eta -:--:--
     ---------------------------------------- 0.1/20.6 MB 1.5 MB/s eta 0:00:14
      --------------------------------------- 0.3/20.6 MB 4.2 MB/s eta 0:00:05
     - -------------------------------------- 0.8/20.6 MB 5.5 MB/s eta 0:00:04
     -- ------------------------------------- 1.2/20.6 MB 6.7 MB/s eta 0:00:03
     --- ------------------------------------ 1.6/20.6 MB 7.1 MB/s eta 0:00:03
     --- ------------------------------------ 1.9/20.6 MB 7.3 MB/s eta 0:00:03
     ---- ----------------------------------- 2.3/20.6 MB 7.5 MB/s eta 0:00:03
     ----- ---------------------------------- 2.7/20.6 MB 7.5 MB/s eta 0:00:03
     ------ --------------------------------- 3.1/20.6 MB 7.6 MB/s eta 0:00:03
     ------ --------------------------------- 3.5/20.6 MB 7.8 MB/s eta 0:00:03
     ------- -------------------------------- 3.9/20.6 MB 7.8 MB/s eta 0:00:03
     -------- ------------------------------- 4.3/20.6 MB 7

In [4]:
import pycaret
import sklearn
print(pycaret.__version__)
print(sklearn.__version__)

3.2.0
1.3.0


## Importación de datos

In [5]:
# read csv file
import pandas as pd
import numpy as np

data = pd.read_csv('data\AirPassengers.csv')
data['Date'] = pd.to_datetime(data['Date'])
data.head()

,Date,Passengers
0,1949-01-01,112
1,1949-02-01,118
2,1949-03-01,132
3,1949-04-01,129
4,1949-05-01,121


In [6]:
data.head()

,Date,Passengers
0,1949-01-01,112
1,1949-02-01,118
2,1949-03-01,132
3,1949-04-01,129
4,1949-05-01,121


## Análisis de datos

In [7]:
# create 12 month moving average
data['MA12'] = data['Passengers'].rolling(12).mean()
data

,Date,Passengers,MA12
0,1949-01-01,112,NaN
1,1949-02-01,118,NaN
2,1949-03-01,132,NaN
3,1949-04-01,129,NaN
4,1949-05-01,121,NaN
...,...,...,...
139,1960-08-01,606,463.333333
140,1960-09-01,508,467.083333
141,1960-10-01,461,471.583333
142,1960-11-01,390,473.916667


In [8]:
# plot the data and MA
import plotly.express as px
fig = px.line(data, x="Date", y=["Passengers", "MA12"], template = 'plotly_dark')
fig.show()

## Transformación de datos

In [9]:
# extract month and year from dates
data['Month'] = [i.month for i in data['Date']]
data['Year'] = [i.year for i in data['Date']]

# create a sequence of numbers
data['Series'] = np.arange(1,len(data)+1)

# drop unnecessary columns and re-arrange
data.drop(['Date', 'MA12'], axis=1, inplace=True)
data = data[['Series', 'Year', 'Month', 'Passengers']] 

# check the head of the dataset
data.head()

,Series,Year,Month,Passengers
0,1,1949,1,112
1,2,1949,2,118
2,3,1949,3,132
3,4,1949,4,129
4,5,1949,5,121


### División de datos en train y test

In [10]:
# split data into train-test set
train = data[data['Year'] < 1960]
test = data[data['Year'] >= 1960]
# check shape
train.shape, test.shape


((132, 4), (12, 4))

### Preprocesamiento de datos

In [13]:
# import the regression module
from pycaret.regression import *

# initialize setup
s = setup(data = train, test_data = test, target = 'Passengers',
          fold_strategy = 'timeseries', data_split_shuffle = False, fold_shuffle = False, numeric_features = ['Year', 'Series'],
          fold = 3, transform_target = True, session_id = 123)

,Description,Value
0,Session id,123
1,Target,Passengers
2,Target type,Regression
3,Original data shape,"(144, 4)"
4,Transformed data shape,"(144, 4)"
5,Transformed train set shape,"(132, 4)"
6,Transformed test set shape,"(12, 4)"
7,Numeric features,2
8,Preprocess,True
9,Imputation type,simple


# Parte II

## Entrenamiento del modelo de regresión

In [14]:
best = compare_models(sort = 'MAE')

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
omp,Orthogonal Matching Pursuit,44.3192,3050.6952,53.8852,-0.4522,0.1714,0.1521,0.0133
br,Bayesian Ridge,45.5643,3233.7133,55.6766,-0.6553,0.1765,0.1576,0.0133
ridge,Ridge Regression,45.7835,3257.2206,55.8542,-0.6687,0.1770,0.1585,0.8800
lar,Least Angle Regression,45.8053,3260.6009,55.8841,-0.6721,0.1771,0.1586,0.8867
lr,Linear Regression,45.8353,3263.6600,55.9128,-0.6739,0.1772,0.1587,1.3933
huber,Huber Regressor,46.0420,3308.9724,56.4600,-0.8186,0.1802,0.1618,0.0167
lasso,Lasso Regression,51.7106,3939.2790,61.4955,-1.4896,0.2374,0.1830,0.8900
llar,Lasso Least Angle Regression,51.7106,3939.2790,61.4955,-1.4896,0.2374,0.1830,0.8633
gbr,Gradient Boosting Regressor,52.4896,4355.4048,63.7434,-0.7220,0.2142,0.1618,0.0233
en,Elastic Net,52.5924,3937.1760,61.2301,-1.4778,0.2367,0.1870,0.8833


In [15]:
prediction_holdout = predict_model(best)

# Parte III

## Predicción de series temporales

In [16]:
# generate predictions on the original dataset
predictions = predict_model(best, data=data)

# add a date column in the dataset
predictions['Date'] = pd.date_range(start='1949-01-01', end = '1960-12-01', freq = 'MS')


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Orthogonal Matching Pursuit,33.2810,2048.6856,45.2624,0.8567,0.1356,0.1126


### Visualización de resultados

In [17]:
# line plot
fig = px.line(predictions, x='Date', y=["Passengers", "Label"], template = 'plotly_dark')
# add a vertical rectange for test-set separation
fig.add_vrect(x0="1960-01-01", x1="1960-12-01", fillcolor="grey", opacity=0.25, line_width=0)
fig.show()

ValueError: All arguments should have the same length. The length of argument `y` is 2, whereas the length of  previously-processed arguments ['Date'] is 144

In [18]:
final_best = finalize_model(best)

## Predicción con nuevos datos

In [19]:
future_dates = pd.date_range(start = '1961-01-01', end = '1965-01-01', freq = 'MS')
future_df = pd.DataFrame()

In [20]:
future_df['Month'] = [i.month for i in future_dates]
future_df['Year'] = [i.year for i in future_dates]    
future_df['Series'] = np.arange(145,(145+len(future_dates)))
future_df.head()

,Month,Year,Series
0,1,1961,145
1,2,1961,146
2,3,1961,147
3,4,1961,148
4,5,1961,149


In [21]:
predictions_future = predict_model(final_best, data=future_df)
predictions_future.head()

,Month,Year,Series,prediction_label
0,1,1961,145,515.565121
1,2,1961,146,520.240543
2,3,1961,147,524.952086
3,4,1961,148,529.699984
4,5,1961,149,534.484468


In [22]:
concat_df = pd.concat([data,predictions_future], axis=0)

concat_df_i = pd.date_range(start='1949-01-01', end = '1965-01-01', freq = 'MS')
concat_df.set_index(concat_df_i, inplace=True)

In [23]:
concat_df.head()

,Series,Year,Month,Passengers,prediction_label
1949-01-01,1,1949,1,112.0,NaN
1949-02-01,2,1949,2,118.0,NaN
1949-03-01,3,1949,3,132.0,NaN
1949-04-01,4,1949,4,129.0,NaN
1949-05-01,5,1949,5,121.0,NaN


In [24]:
fig = px.line(concat_df, x=concat_df.index, y=["Passengers", "Label"], template = 'plotly_dark')
fig.show()

ValueError: All arguments should have the same length. The length of argument `y` is 2, whereas the length of  previously-processed arguments ['index'] is 193